# Week 4 Visualizations: Maps

**Input**: ACLED events + Economics master (sector percentages)  
**Output**: JSON datasets for D3.js charts in viz-datasets/

In [52]:
import pandas as pd
import numpy as np
import json
from pathlib import Path

# For visualizations (optional)
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set style for plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Load Data

In [53]:
raw_data_path = Path('../raw-data')
processed_data_path = Path('../processed-data')
viz_datasets_path = Path('../viz-datasets')

df_econ = pd.read_csv(processed_data_path / 'economics-countries-master.csv')
df_econ = df_econ.rename(columns=str.lower)
df_econ = ( df_econ.sort_values('year').groupby('country').last().reset_index())

print(f"Economics: {len(df_econ):,} country-years")
df_econ.head(10)

Economics: 220 country-years


,country,year,primary_%,secondary_%,tertiary_%,tourism_%,gdp_usd,population,inflation_%,debt_%
0,Afghanistan,2023,41.89,8.90,49.21,NaN,1.426650e+10,41128771.0,2.30,NaN
1,Albania,2023,26.58,18.51,54.91,5.56,1.891638e+10,2777689.0,6.73,82.38
2,Algeria,2023,29.78,22.93,47.29,1.39,1.949984e+11,44903225.0,9.27,NaN
3,Andorra,2023,1.90,12.25,85.85,NaN,3.352031e+09,79824.0,NaN,NaN
4,Angola,2023,46.78,13.37,39.85,NaN,1.067828e+11,35588987.0,25.75,NaN
5,Anguilla,2023,5.20,14.39,80.41,NaN,NaN,NaN,NaN,NaN
6,Antigua and Barbuda,2023,5.49,18.45,76.06,11.71,1.867733e+09,93763.0,7.53,NaN
7,Argentina,2023,12.80,24.03,63.17,1.68,6.311334e+11,46234830.0,NaN,NaN
8,Armenia,2023,15.66,19.63,64.71,NaN,1.951347e+10,2780469.0,8.64,60.08
9,Aruba,2023,4.11,8.70,87.19,21.20,3.544708e+09,106445.0,4.26,NaN


## Viz 1: Colorpleth Map

In [54]:
# Select only the fields you want
df_econ['gdp_per_capita'] = df_econ['gdp_usd'] / df_econ['population']

df_econ_latest = df_econ[['country', 'year', 'primary_%']].copy()
df_econ_latest = df_econ_latest.rename(columns={"primary_%": "primary"})
df_viz2 = df_econ_latest[df_econ_latest['primary'].notna()].copy()
df_viz2['country'] = df_viz2['country'].replace({
    "United States": "United States of America"
})
df_viz2['country'] = df_viz2['country'].replace({
    "Russian Federation": "Russia"
})
df_viz2['country'] = df_viz2['country'].replace({
    "Venezuela (Bolivarian Republic of)": "Venezuela"
})
df_viz2['country'] = df_viz2['country'].replace({
    "Bolivia (Plurinational State of)": "Bolivia"
})
df_viz2['country'] = df_viz2['country'].replace({
    "D.R. of the Congo": "Dem. Rep. Congo"
})
df_viz2['country'] = df_viz2['country'].replace({
    "South Sudan": "S. Sudan"
})
df_viz2['country'] = df_viz2['country'].replace({
    "Central African Republic": "Central African Rep."
})
df_viz2['country'] = df_viz2['country'].replace({
    "Iran (Islamic Republic of)": "Iran"
})
df_viz2['country'] = df_viz2['country'].replace({
    "Türkiye": "Turkey"
})
df_viz2['country'] = df_viz2['country'].replace({
    "Republic of Korea": "South Korea"
})
df_viz2['country'] = df_viz2['country'].replace({
    "D.P.R. of Korea": "North Korea"
})
df_viz2['country'] = df_viz2['country'].replace({
    "Viet Nam": "Vietnam"
})
df_viz2['country'] = df_viz2['country'].replace({
    "Lao People's DR": "Laos"
})
df_viz2['country'] = df_viz2['country'].replace({
    "U.R. of Tanzania: Mainland": "Tanzania"
})
df_viz2['country'] = df_viz2['country'].replace({
    "Dominican Republic": "Dominican Rep."
})
df_viz2['country'] = df_viz2['country'].replace({
    "Equatorial Guinea": "Eq. Guinea"
})
df_viz2['country'] = df_viz2['country'].replace({
    "State of Palestine": "Palestine"
})
df_viz2['country'] = df_viz2['country'].replace({
    "Syrian Arab Republic": "Syria"
})
df_viz2['country'] = df_viz2['country'].replace({
    "Republic of Moldova": "Moldova"
})
df_viz2['country'] = df_viz2['country'].replace({
    "Solomon Islands": "Solomon Is."
})
df_viz2['country'] = df_viz2['country'].replace({
    "Brunei Darussalam": "Brunei"
})
df_viz2['country'] = df_viz2['country'].replace({
    "Bosnia and Herzegovina": "Bosnia and Herz."
})
df_viz2['country'] = df_viz2['country'].replace({
    "North Macedonia": "Macedonia"
})
somaliland_rows = df_viz2[df_viz2['country'] == "Somalia"].copy()
somaliland_rows['country'] = "Somaliland"
df_viz2 = pd.concat([df_viz2, somaliland_rows], ignore_index=True)
cyprus_rows = df_viz2[df_viz2['country'] == "Cyprus"].copy()
cyprus_rows['country'] = "N. Cyprus"
df_viz2 = pd.concat([df_viz2, cyprus_rows], ignore_index=True)


# Metadata
metadata = {
    'title': 'Maps',
    'description': '',
    'source': 'World Bank',
    'date_range': '>= 2024',
    'notes': ''
}

# Create output structure for viz 2
viz1_output = {
    'metadata': metadata,
    'data': df_viz2.to_dict(orient='records')
}

output_file_viz1 = viz_datasets_path / 'viz7_maps.json'
with open(output_file_viz1, 'w', encoding='utf-8') as f:
    json.dump(viz1_output, f, indent=2, ensure_ascii=False)

print(f"\n[OK] Saved VIZ 2: {output_file_viz1.name} ({output_file_viz1.stat().st_size / 1024:.1f} KB)")
print(f"Total countries: {len(df_viz2):,}")


[OK] Saved VIZ 2: viz7_maps.json (18.5 KB)
Total countries: 219
